In [ ]:
# %%capture --no-stderr
# %pip install -U tavily-python langchain_community


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# import os

# TAVILY_API_KEY = "<TAVILY_API_KEY>"

# os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY

# import getpass
# import os

# if not os.environ.get("TAVILY_API_KEY"):
#     os.environ["TAVILY_API_KEY"] = getpass.getpass("Tavily API key:\n")

In [ ]:
# %pip install -U langchain langchain-openai

# LANGSMITH_TRACING=true
# LANGSMITH_ENDPOINT="https://api.smith.langchain.com"
# LANGSMITH_API_KEY="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"
# LANGSMITH_PROJECT="pr-shadowy-clay-24"
# OPENAI_API_KEY="<your-openai-api-key>"

   ---------------------------------------- 0.0/567.4 kB ? eta -:--:--
   ---------------------------------------- 0.0/567.4 kB ? eta -:--:--
   ------------------ --------------------- 262.1/567.4 kB ? eta -:--:--
   ---------------------------------------- 567.4/567.4 kB 1.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
